In [1]:
import arviz_base as azb
import pandas as pd
import bambi as bmb
SEED = 91764

In [2]:
trolley = pd.read_csv("https://raw.githubusercontent.com/rmcelreath/rethinking/master/data/Trolley.csv", sep=";")
trolley = trolley[["response", "action", "intention", "contact"]][::7]
trolley["action"] = pd.Categorical(trolley["action"], ordered=False)
trolley["intention"] = pd.Categorical(trolley["intention"], ordered=False)
trolley["contact"] = pd.Categorical(trolley["contact"], ordered=False)
trolley["response"] = pd.Categorical(trolley["response"], ordered=True)

In [3]:
model = bmb.Model(
    "response ~ 0 + action + intention + contact", 
    
    data=trolley, 
    family="cumulative"
)
idata = model.fit(random_seed=SEED)
model.predict(idata, kind="response", random_seed=SEED, inplace=True)
del idata.posterior
del idata.sample_stats
azb.convert_to_datatree(idata).to_netcdf("../prerun/dt_ord.nc")

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [threshold, action, intention, contact]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 11 seconds.


In [4]:
import matplotlib.pyplot as plt
import bambi as bmb
import numpy as np
import pandas as pd

length = [
    1.3, 1.32, 1.32, 1.4, 1.42, 1.42, 1.47, 1.47, 1.5, 1.52, 1.63, 1.65, 1.65, 1.65, 1.65,
    1.68, 1.7, 1.73, 1.78, 1.78, 1.8, 1.85, 1.93, 1.93, 1.98, 2.03, 2.03, 2.31, 2.36, 2.46,
    3.25, 3.28, 3.33, 3.56, 3.58, 3.66, 3.68, 3.71, 3.89, 1.24, 1.3, 1.45, 1.45, 1.55, 1.6, 
    1.6, 1.65, 1.78, 1.78, 1.8, 1.88, 2.16, 2.26, 2.31, 2.36, 2.39, 2.41, 2.44, 2.56, 2.67, 
    2.72, 2.79, 2.84
]
choice = [
    "I", "F", "F", "F", "I", "F", "I", "F", "I", "I", "I", "O", "O", "I", "F", "F", 
    "I", "O", "F", "O", "F", "F", "I", "F", "I", "F", "F", "F", "F", "F", "O", "O", 
    "F", "F", "F", "F", "O", "F", "F", "I", "I", "I", "O", "I", "I", "I", "F", "I", 
    "O", "I", "I", "F", "F", "F", "F", "F", "F", "F", "O", "F", "I", "F", "F"
]

sex = ["Male"] * 32 + ["Female"] * 31
data = pd.DataFrame({"choice": choice, "length": length, "sex": sex})
data["choice"]  = pd.Categorical(
    data["choice"].map({"I": "Invertebrates", "F": "Fish", "O": "Other"}), 
    ["Other", "Invertebrates", "Fish"], 
    ordered=True
)

model = bmb.Model("choice ~ length + sex", data, family="categorical")
idata = model.fit(random_seed=SEED)
model.predict(idata, kind="response", random_seed=SEED, inplace=True)
del idata.posterior
del idata.sample_stats
azb.convert_to_datatree(idata).to_netcdf("../prerun/dt_cat.nc")

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Intercept, length, sex]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 3 seconds.
